<a href="https://colab.research.google.com/github/newmantic/auto_callable_option/blob/main/auto_callable_option.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

class AutoCallableOption:
    def __init__(self, initial_price, strike_price, call_barrier, maturity, coupon_rate, risk_free_rate, volatility):
        self.initial_price = initial_price
        self.strike_price = strike_price
        self.call_barrier = call_barrier
        self.maturity = maturity
        self.coupon_rate = coupon_rate
        self.risk_free_rate = risk_free_rate
        self.volatility = volatility

    def simulate_price_path(self, n_steps, n_scenarios):
        dt = self.maturity / n_steps
        price_paths = np.zeros((n_steps + 1, n_scenarios))
        price_paths[0] = self.initial_price

        for t in range(1, n_steps + 1):
            z = np.random.standard_normal(n_scenarios)
            price_paths[t] = price_paths[t - 1] * np.exp((self.risk_free_rate - 0.5 * self.volatility ** 2) * dt + self.volatility * np.sqrt(dt) * z)

        return price_paths

    def calculate_payoff(self, price_paths):
        for t in range(1, price_paths.shape[0]):
            if np.all(price_paths[t] >= self.call_barrier):
                # Option is called, and coupon is paid
                return self.coupon_rate * t / price_paths.shape[0]

        # Option not called, payoff at maturity
        final_price = price_paths[-1]
        return max(final_price - self.strike_price, 0)

    def run_simulation(self, n_steps=252, n_scenarios=10000):
        price_paths = self.simulate_price_path(n_steps, n_scenarios)
        payoffs = np.zeros(n_scenarios)

        for i in range(n_scenarios):
            payoffs[i] = self.calculate_payoff(price_paths[:, i])

        return np.mean(payoffs)

# Example usage
if __name__ == "__main__":
    option = AutoCallableOption(
        initial_price=100,
        strike_price=110,
        call_barrier=105,
        maturity=1,
        coupon_rate=0.05,
        risk_free_rate=0.02,
        volatility=0.2
    )

    expected_payoff = option.run_simulation()
    print(f"Expected Payoff: {expected_payoff:.2f}")

Expected Payoff: 0.01
